In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp "drive/My Drive/Colab Notebooks/all.zip" "all.zip"

In [8]:
import os
import zipfile

local_zip = 'all.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

local_zip = '/tmp/train.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/cat_dog_data')
zip_ref.close()

local_zip = '/tmp/test1.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/cat_dog_data')
zip_ref.close()
!ls "/tmp/cat_dog_data"

test1  train


In [0]:
import numpy as np
import pandas as pd
import glob
import cv2
import os
import matplotlib.pyplot as plt

In [0]:
# set data path
base_dir = '/tmp/cat_dog_data'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir,'test1')

In [11]:
train_files = np.array(list(map(os.path.basename,glob.glob(train_dir + '/*'))))
test_files = np.array(list(map(os.path.basename,glob.glob(test_dir + '/*'))))
sample = os.path.join(train_dir,train_files[0])
print('train data is {} file {} with shape {}'.format(train_files.shape,train_files[0],
                                                   cv2.imread(sample).shape))

train data is (25000,) file dog.2877.jpg with shape (160, 177, 3)


In [12]:
def get_tag_string(path):
    base = os.path.basename(path)
    if 'cat' in base:
        return 0
    return 1
get_tag_string(train_dir+'/cat.1.jpg')

0

In [0]:
train_df = pd.DataFrame({'id':train_files})
train_df['label'] = train_df['id'].apply(get_tag_string)

train_df.head()

,id,label
0,dog.12415.jpg,1
1,cat.11107.jpg,0
2,dog.10305.jpg,1
3,cat.5086.jpg,0
4,dog.9881.jpg,1


In [13]:
test_df = pd.DataFrame({ 'id': pd.Series(range(1,len(test_files)+1),index=range(len(test_files)))})
test_df['id'] = test_df['id'].apply(lambda x: str(x)+'.jpg')
test_df.head()


,id
0,1.jpg
1,2.jpg
2,3.jpg
3,4.jpg
4,5.jpg


从网络下载training图片。下面是真正的训练

In [0]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D,Dropout,Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [0]:
drive_path = 'drive/My Drive/Colab Notebooks/' #save to drive
bottleneck_path = drive_path + 'bottleneck/'

In [0]:
# !pip uninstall keras-preprocessing 
# !pip install git+https://github.com/keras-team/keras-preprocessing.git 
# %%!y

In [0]:
#id + label + bottleneck
train_df.to_csv(drive_path+'train.csv',index=False)


In [0]:
from keras_preprocessing.image import ImageDataGenerator
batch_size = 100
datagen = ImageDataGenerator(rescale=1./255)


In [0]:
def gen_bottleneck(model,name_weight):
 
  generator = datagen.flow_from_dataframe(dataframe=train_df,
                                        directory=train_dir,
                                        x_col='id',
                                        y_col=None,
                                        classes=None,
                                        batch_size=batch_size,
                                        class_mode=None,
                                        target_size=(299,299),
                                        shuffle=False)
    
  bottleneck_features = model.predict_generator(generator, train_df.shape[0]/batch_size)
  with open(name_weight+'train.npy','wb') as f:
    np.save(f,bottleneck_features)
  print('saved train.npy')

   

  generator = datagen.flow_from_dataframe(dataframe=test_df,
                                           directory=test_dir,
                                            x_col='id',
                                            y_col=None,
                                            classes=None,
                                            batch_size=batch_size,
                                            class_mode=None,
                                            target_size=(299,299),
                                            shuffle=False)
  bottleneck_features = model.predict_generator(generator, test_df.shape[0]/batch_size)
  with open(name_weight+'test.npy','wb') as f:
    np.save(f,bottleneck_features)
    

In [19]:
#input 299*299*3
model = Sequential()
model.add(Xception(include_top=False,weights='imagenet'))
model.add(GlobalAveragePooling2D())
gen_bottleneck(model, bottleneck_path+'xcept/')


Found 25000 images.
saved train.npy
Found 12500 images.


In [20]:
model = Sequential()
model.add(InceptionV3(include_top=False,weights='imagenet'))
model.add(GlobalAveragePooling2D())
gen_bottleneck(model, bottleneck_path+'incept/')

87916544/87910968 [==============================] - 1s 0us/step
Found 25000 images.
saved train.npy
Found 12500 images.
